## Requires isochroner module, see dfsnow GitHub

In [8]:
import isochroner as iso
import geopandas as gpd
import pandas as pd
import iso_keys as keys
import fiona
import os
from shapely import wkt, geometry

In [6]:
# Use isochroner to create isochrones from the specified shapefile
gdf = gpd.read_file(os.path.join('shapefiles', 'ti_2015_chi_tracts.shp'))
iso.isochrone_batch(
    gdf,
    key=keys.api_key,
    out_filename=os.path.join('data', 'isochrones.csv'),
    duration=20,
    keep_cols=['GEOID'],
    batch_size=3)

In [9]:
# Function to convert query output to an ESRI shapefile
def isochrones_to_shp(df, filename, crs, format='ESRI Shapefile'):
    """
    Convert a dataframe of isochrones to a shapefile or GeoJSON

    :param df: Input dataframe
    :param key: Google Maps API key passed within function
    :param filename: Filename to save as
    :param crs: CRS to use when creating the output
    :param format: Format to use for output, geojson or esri
    :return: Shapefile or GeoJSON with attached duration and origin point data
    """
    df['geometry'] = df['geometry'].astype(str).map(wkt.loads)
    crs = {'init': 'epsg:' + str(crs)}
    df = gpd.GeoDataFrame(df, crs=crs, geometry=df['geometry'])

    iso_schema = {
        'geometry': 'Polygon',
        'properties': {'id': 'int',
                       'GEOID': 'str',
                       'coords': 'str',
                       'duration': 'int'}
    }

    with fiona.open(filename, 'w', format, iso_schema) as c:
        for index, geo in df.iterrows():
            c.write({
                'geometry': geometry.mapping(geo['geometry']),
                'properties': {'id': index,
                               'GEOID': str(geo['GEOID']),
                               'coords': str(geo['coords']),
                               'duration': int(geo['duration'])}
            })

df = pd.read_csv(os.path.join('data', 'isochrones.csv'))
isochrones_to_shp(df, os.path.join('shapefiles', 'isochrones.shp'), crs=4269)